In [1]:
import sys
sys.path.append("..")
from MPRA_predict.utils import *
from MPRA_predict.datasets import *
from MPRA_predict.metrics import *

from collections import Counter
import re

In [2]:
df = pd.read_csv('../data/Sei/Sei_tracks_info.csv')

In [3]:
df_pivot = df.pivot_table(
    values="index", 
    index="cell_type", 
    columns="assay", 
    aggfunc=list,
)

df_pivot

assay                    ADNP AEBP2 AFF1 AFF4 AGO1 AGO2  AHR AHRR AP-2alpha AP-2gamma APOBEC3B   AR ARID1A ARID2 ARID3A ARID5B ARNT ARNT2 ARNTL ARRB1 ASCL1 ASCL2 ASH1L ASH2L ASXL1 ATAC-seq ATF1 ATF2 ATF3 ATF4 ATF5 ATF7IP ATOH1 ATRX AUTS2 BACH1 BACH2 BAF155 BAF170 BAHD1 BANF1 BANP BARX1 BARX2 BATF BATF3 BCAT1 BCL11A BCL11B BCL3 BCL6 BCLAF1 BCOR BDP1 BHLHE40 BIRA BMI1 BPTF BRAF BRCA1 BRD1 BRD2    BRD3    BRD4 BRD7 BRD9 BRDU BRF1 BRF2 BRPF3 BTAF1 Bach1 Brg1 C11orf30 C17orf49 C17orf96 CAMTA2 CASP8AP2 CBFA2T2 CBFB CBX1 CBX2 CBX3 CBX4 CBX5 CBX6 CBX7 CBX8 CCDC101 CCND2 CCNT2 CD59 CD74 CDC5L CDCA2 CDK12 CDK2 CDK6 CDK7 CDK8 CDK9 CDX2 CEBPA CEBPB CEBPD CEBPG CEBPZ CENPA CENPC CENPT CHAMP1 CHAT CHD1 CHD2 CHD4 CHD5 CHD7 CHD8 CHRM2 CIITA CLOCK CNOT3 COPS2 COREST CPSF3L CREB1 CREB3 CREB3L4 CREBBP CREM CRY1 CSDE1 CSNK2A1 CTBP1 CTBP2 CTCF CTCFL CTNNB1 CUX1 CtBP2 DACOR1 DAXX DCP1A DDX20 DDX21 DDX5 DEAF1  DEK DIDO1 DLX1 DLX2 DMC1 DNMT1 DNMT3A DNMT3B    DNase DNase.all.peaks DNase.fdr0.01.hot  \
cell_type                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      
1015c                     NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN       NaN       NaN      NaN  NaN    NaN   NaN    NaN    NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN      NaN  NaN  NaN  NaN  NaN  NaN    NaN   NaN  NaN   NaN   NaN   NaN    NaN    NaN   NaN   NaN  NaN   NaN   NaN  NaN   NaN   NaN    NaN    NaN  NaN  NaN    NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN   NaN  NaN  NaN     NaN  [8996]  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN  NaN      NaN      NaN      NaN    NaN      NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   NaN   NaN  NaN  NaN   NaN   NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   NaN     NaN    NaN  NaN  NaN   NaN     NaN   NaN   NaN  NaN   NaN    NaN  NaN   NaN    NaN  NaN   NaN   NaN   NaN  NaN   NaN  NaN   NaN  NaN  NaN  NaN   NaN    NaN    NaN      NaN             NaN               NaN   
10326                     NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN       NaN       NaN      NaN  NaN    NaN   NaN    NaN    NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN      NaN  NaN  NaN  NaN  NaN  NaN    NaN   NaN  NaN   NaN   NaN   NaN    NaN    NaN   NaN   NaN  NaN   NaN   NaN  NaN   NaN   NaN    NaN    NaN  NaN  NaN    NaN  NaN  NaN     NaN  NaN  NaN  NaN  NaN   NaN  NaN  NaN  [8997]     NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN  NaN      NaN      NaN      NaN    NaN      NaN     NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN     NaN   NaN   NaN  NaN  NaN   NaN   NaN   NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN    NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN   NaN   NaN   NaN   NaN   NaN    NaN    NaN   NaN   NaN     NaN    NaN  NaN  NaN   NaN     NaN   NaN   NaN  NaN   NaN    NaN  NaN   NaN    NaN  NaN   NaN   NaN   NaN  NaN   NaN  NaN   NaN  NaN  NaN  NaN   NaN    NaN    NaN      NaN             NaN               NaN   
1184_Stem_cell            NaN   

In [16]:
# assays = ['DNase', 'H3K4me3', 'H3K27ac', 'CTCF']

assays = ['DNase', 'H3K4me3', 'H3K27ac', 'H3K27me3', 'H3K4me1', 'H3K36me3', 'H3K9me3', 'CTCF']

cell_types = ['K562', 'HepG2']
cell_types_standard_names = ['K562_Leukemia_Cell', 'HepG2_Hepatocellular_Carcinoma']

In [17]:
df_track = df_pivot.loc[cell_types_standard_names][assays]
df_track.map(lambda x: len(x) if isinstance(x, list) else 0)

assay,DNase,H3K4me3,H3K27ac,H3K27me3,H3K4me1,H3K36me3,H3K9me3,CTCF
cell_type,,,,,,,,
K562_Leukemia_Cell,58,91,51,34,11,5,29,29
HepG2_Hepatocellular_Carcinoma,9,8,9,2,4,3,2,17


In [18]:
####### Agarwal_MPRA

# pred_array = load_h5('outputs/Agarwal_MPRA_Sei_pred.h5')
pred_array = load_h5('outputs/Agarwal_MPRA_Sei_pred.h5')
print(pred_array.shape)

(56975, 21907)


In [19]:
MPRA_df = pd.read_csv('../data/Agarwal_MPRA/Agarwal_MPRA_joint_56k.csv')

print(MPRA_df.shape)

(56975, 5)


In [44]:
pearson_df = pd.DataFrame(index=cell_types, columns=assays, dtype=float)

for i, cell_type in enumerate(cell_types):
    for j, assay in enumerate(assays):
        indice = df_track.loc[cell_types_standard_names[i], assay]

        if isinstance(indice, list) and len(indice) > 0:
            for idx in indice:
                pred = logit(pred_array[:, idx])
                true = MPRA_df[cell_type]
                r, _ = pearson(pred, true)
                print(cell_type, assay, idx, r)

            pred = logit(pred_array[:, indice], eps=0).mean(1)
            true = MPRA_df[cell_type]
            r, _ = pearson(pred, true)
            pearson_df.loc[cell_type, assay] = r
            MPRA_df[f'{cell_type}_{assay}'] = pred
        else:
            MPRA_df[f'{cell_type}_{assay}'] = np.nan

pearson_df

K562 DNase 1490 0.23634473653657687
K562 DNase 2919 0.40183563461205385
K562 DNase 3167 0.4763686683163797
K562 DNase 4632 0.47667583220975873
K562 DNase 4633 0.4606272499084298
K562 DNase 10715 0.4648229103227851
K562 DNase 10716 0.47764334570991696
K562 DNase 10717 0.4661945889310495
K562 DNase 10718 0.39706811401439435
K562 DNase 10719 0.4802493056316183
K562 DNase 10720 0.4408137361556317
K562 DNase 10721 0.4160031370508047
K562 DNase 10722 0.4362234353842961
K562 DNase 10723 0.4538683437762553
K562 DNase 10724 0.43779807097886386
K562 DNase 10725 0.4755018546580194
K562 DNase 10726 0.43351294262982787
K562 DNase 10727 0.4647490313778555
K562 DNase 10728 0.5458300545071509
K562 DNase 10729 0.4231547849437789
K562 DNase 10730 0.39867679055624733
K562 DNase 10731 0.4336591843523095
K562 DNase 10732 0.4516785083269676
K562 DNase 10749 0.4389411587992431
K562 DNase 10817 0.4202258204349552
K562 DNase 10822 0.5574108601842458
K562 DNase 10857 0.48932209432116447
K562 DNase 10954 0.43775

,DNase,H3K4me3,H3K27ac,CTCF
K562,0.450,0.459,0.426,0.076
HepG2,0.334,0.339,0.352,-0.034


In [45]:
pearson_df = pd.DataFrame(index=cell_types, columns=assays, dtype=float)

for i, cell_type in enumerate(cell_types):
    for j, assay in enumerate(assays):
        pred = MPRA_df[f'{cell_type}_{assay}']
        true = MPRA_df[cell_type]
        r, _ = pearson(pred, true)
        pearson_df.loc[cell_type, assay] = r

pearson_df

,DNase,H3K4me3,H3K27ac,CTCF
K562,0.450,0.459,0.426,0.076
HepG2,0.334,0.339,0.352,-0.034


In [46]:
# 真实活性和DNase的相关性
cell_types = ['K562', 'HepG2']
assays = ['DNase', 'H3K4me3', 'H3K27ac', 'H3K27me3', 'H3K4me1', 'H3K36me3', 'H3K9me3', 'CTCF']

for assay in assays:
    cell_types_assay = [f'{cell_type}_{assay}' for cell_type in cell_types]
    pearson_df = pd.DataFrame(index=cell_types, columns=cell_types_assay, dtype=float)
    for cell_type in cell_types:
        for cell_type_assay in cell_types_assay:
            df = MPRA_df
            x, y = df[f'{cell_type}'], df[f'{cell_type_assay}']
            r, p = pearson(x, y)
            pearson_df.loc[cell_type, cell_type_assay] = r
    print(assay)
    print(pearson_df)

DNase
       K562_DNase  HepG2_DNase
K562        0.450        0.101
HepG2       0.247        0.334
H3K4me3
       K562_H3K4me3  HepG2_H3K4me3
K562          0.459          0.164
HepG2         0.320          0.339
H3K27ac
       K562_H3K27ac  HepG2_H3K27ac
K562          0.426          0.028
HepG2         0.353          0.352
H3K27me3
       K562_H3K27me3  HepG2_H3K27me3
K562           0.045          -0.010
HepG2          0.082          -0.011
H3K4me1
       K562_H3K4me1  HepG2_H3K4me1
K562          0.334         -0.028
HepG2         0.285          0.314
H3K36me3
       K562_H3K36me3  HepG2_H3K36me3
K562           0.295           0.184
HepG2          0.293           0.166
H3K9me3
       K562_H3K9me3  HepG2_H3K9me3
K562          0.281         -0.024
HepG2         0.279          0.016
CTCF
       K562_CTCF  HepG2_CTCF
K562       0.076       0.002
HepG2     -0.051      -0.034


In [47]:
# normalize between different cell types and assays

norm_params = pd.read_csv('outputs/Sei_feature_norm_parameters.csv', index_col=0)

for i, cell_type in enumerate(cell_types):
    for j, assay in enumerate(assays):
        
        x = MPRA_df[f'{cell_type}_{assay}']

        x_mean = norm_params.loc['Gosai_MPRA_train', f'{cell_type}_{assay}_mean']
        x_std = norm_params.loc['Gosai_MPRA_train', f'{cell_type}_{assay}_std']
        print(x_mean, x_std)
        x = (x - x_mean) / x_std
        
        ######
        x[x<0] = 0
        ######

        MPRA_df[f'{cell_type}_{assay}'] = x

-6.417223285308433 1.9469177478992024
-8.963800672911345 1.6842842596202077
-8.37802168987605 1.1557352256723057


KeyError: 'K562_H3K27me3_mean'

In [43]:
MPRA_df

,seq,chr,HepG2,K562,WTC11,K562_DNase,K562_H3K4me3,K562_H3K27ac,K562_H3K27me3,K562_H3K4me1,K562_H3K36me3,K562_H3K9me3,K562_CTCF,HepG2_DNase,HepG2_H3K4me3,HepG2_H3K27ac,HepG2_H3K27me3,HepG2_H3K4me1,HepG2_H3K36me3,HepG2_H3K9me3,HepG2_CTCF
0,AGGACCGGATCAACTGTAGGACCAGGCTCTGCCTCCTTCTCTCTCC...,chr1,0.633,0.080,-0.387,1.528,2.065,1.971,-6.207,-5.281,-8.028,-3.572,0.949,1.241,1.943,1.526,-4.622,-6.464,-7.187,-3.427,0.865
1,AGGACCGGATCAACTTGTACTTGCCTTAGGAGGTCAAACACATTAG...,chr10,-1.157,-1.147,-1.507,0.599,0.946,0.564,-6.399,-6.249,-8.733,-6.049,0.563,0.667,0.920,0.727,-4.510,-6.444,-7.940,-3.510,0.635
2,AGGACCGGATCAACTTAGCTCAACACAAATCCATCACTGAAAGCAG...,chr10,-0.066,0.186,0.655,3.294,1.199,0.505,-7.608,-6.247,-7.947,-5.339,5.599,2.986,1.780,1.271,-7.291,-6.640,-8.013,-5.124,5.660
3,AGGACCGGATCAACTGACCCTAAATCAGTATGGGGAAAGGTCCCCA...,chr10,0.846,1.212,-0.637,1.940,1.726,2.129,-6.507,-4.444,-8.702,-5.328,1.059,2.013,1.823,2.210,-5.681,-5.088,-8.140,-3.958,1.090
4,AGGACCGGATCAACTGTGATCTCAGCTTACTGCAACCTCTGCCTCC...,chr10,-0.784,-0.630,-0.612,0.675,0.000,0.000,-8.398,-8.754,-10.251,-5.086,3.078,0.000,0.000,0.000,-9.262,-9.425,-9.639,-4.946,3.059
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56970,AGGACCGGATCAACTCCAAGTCTGCCATCGTGTTCAGAAGGGCCAG...,chrnan,-1.192,-1.080,-1.680,0.575,1.192,1.006,-6.592,-6.735,-9.013,-5.632,0.415,0.216,1.027,0.824,-4.938,-7.513,-8.039,-3.846,0.251
56971,AGGACCGGATCAACTCCAAGTCTGCCATCGTGTTCAGAAGGGCCAG...,chrnan,-0.582,-1.094,-1.637,1.085,1.397,1.176,-7.062,-5.850,-9.326,-6.348,0.142,0.951,1.414,1.144,-6.196,-5.874,-9.238,-3.973,0.170
56972,AGGACCGGATCAACTCCAAGTCTGCCATCGTGTTCATGTTTGCTTT...,chrnan,-0.855,-0.554,-1.083,0.909,1.161,1.147,-7.050,-5.873,-8.751,-6.727,0.689,3.845,2.240,2.768,-6.771,-3.316,-9.631,-5.968,1.846
56973,AGGACCGGATCAACTCCAAGTCTGCCATCGTGTTCAGAAGGGCCAG...,chrnan,-1.231,-0.927,-1.606,0.035,0.370,0.180,-7.004,-7.649,-9.706,-6.395,0.000,0.206,0.611,0.546,-5.795,-7.164,-8.870,-3.975,0.026


In [41]:
pearson_df = pd.DataFrame(index=cell_types, columns=assays, dtype=float)

for i, cell_type in enumerate(cell_types):
    for j, assay in enumerate(assays):
        pred = MPRA_df[f'{cell_type}_{assay}']
        true = MPRA_df[cell_type]
        r, _ = pearson(pred, true)
        pearson_df.loc[cell_type, assay] = r

pearson_df

,DNase,H3K4me3,H3K27ac,CTCF
K562,0.450,0.460,0.426,0.072
HepG2,0.331,0.339,0.352,-0.038


In [42]:
MPRA_df.to_csv('../data/Agarwal_MPRA/Agarwal_MPRA_Sei_feature_norm.csv', index=False)